In [1]:
!pip install -q -U transformers accelerate bitsandbytes
!pip install -q networkx tqdm pyyaml
!pip install flash-attn --no-build-isolation

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 69.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.8.3-cp312-cp312-linux_x86_64.whl size=253780426 sha256=4e2f9e39313266b1544b68138b15b91ee6221eccf14f7902b7c6620351340810
  Stored in directory: /root/.cache/pip/wheels/3d/59/46/f282c12c73dd4bb3c2e3fe199f1a0d0f8cec06df0cccfeee27
Successfully built flash-attn


In [2]:
import os
import json
import re
import time
import yaml
import torch
import gc
import pandas as pd
import networkx as nx
from typing import List, Tuple, Union
from tqdm import tqdm
from networkx.readwrite import json_graph
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [26]:
import torch
import textwrap
from transformers import AutoTokenizer, AutoModelForCausalLM
import re
from typing import Tuple, List
import unicodedata
import pandas as pd
from tqdm import tqdm
from typing import List, Dict
from google.colab import files

## 1. Model configuration

### 1.1. Model definition

In [3]:
class LLM:
    def __init__(
        self,
        model_id: str = "Qwen/Qwen2.5-3B-Instruct",
        use_quantization: bool = False
    ):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model_id = model_id

        print(f">> 📥 Loading Model: {model_id}...")
        print(f">> 🚀 Hardware Optimization: A100 Mode (Flash Attention 2 + bfloat16)")

        load_kwargs = {
            "device_map": "auto",
            "trust_remote_code": True,
            "attn_implementation": "flash_attention_2",
            "torch_dtype": torch.bfloat16,
        }

        if use_quantization:
            print(">> ⚠️ Quantization Enabled (Slower but saves VRAM)")
            load_kwargs["quantization_config"] = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.bfloat16
            )

        try:
            self.tokenizer = AutoTokenizer.from_pretrained(
                model_id,
            )

            self.tokenizer.padding_side = "left"
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token

            self.model = AutoModelForCausalLM.from_pretrained(
                model_id,
                **load_kwargs
            )
            print(">> ✅ Model Loaded Successfully!")

            if self.device == "cuda":
                print(f">> 💾 VRAM Used: {torch.cuda.memory_allocated() / 1024**3:.2f} GiB")

        except Exception as e:
            raise RuntimeError(f"❌ Failed to load model: {e}")

    def generate_response(self, input_text: str, system_prompt: str = "You are a helpful AI assistant.") -> str:
        """Sinh văn bản cho 1 input duy nhất (Dùng để test)"""
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": input_text}
        ]

        text = self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        model_inputs = self.tokenizer([text], return_tensors="pt").to(self.device)

        with torch.no_grad():
            generated_ids = self.model.generate(
                **model_inputs,
                max_new_tokens=512,
                temperature=0.1,
                do_sample=True,
                top_p=0.9,
                repetition_penalty=1.05
            )

        generated_ids = [
            output_ids[len(input_ids):]
            for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]

        return self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    def generate_batch(self, inputs: List[str], system_prompt: str) -> List[str]:
        """
        Sinh văn bản cho nhiều input cùng lúc (Tối ưu cho A100)
        """
        if not inputs: return []

        prompts = []
        for text in inputs:
            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": text}
            ]
            prompts.append(
                self.tokenizer.apply_chat_template(
                    messages, tokenize=False, add_generation_prompt=True
                )
            )

        # Tokenize & Padding
        model_inputs = self.tokenizer(
            prompts,
            return_tensors="pt",
            padding=True,       # Padding tự động
            truncation=True
        ).to(self.device)

        with torch.no_grad():
            generated_ids = self.model.generate(
                **model_inputs,
                max_new_tokens=1024,
                temperature=0.15,
                do_sample=True,
                top_p=0.9,
                repetition_penalty=1.05
            )

        # Cắt bỏ phần input (prompt) khỏi output để lấy câu trả lời
        generated_ids = [
            output_ids[len(input_ids):]
            for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]

        return self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    def unload_model(self):
        print(f">> 🗑️ Unloading model {self.model_id}...")
        if hasattr(self, 'model'): del self.model
        if hasattr(self, 'tokenizer'): del self.tokenizer
        gc.collect()
        if torch.cuda.is_available(): torch.cuda.empty_cache()
        print(">> ✅ Memory cleared!")

### 1.2. Model usage

In [6]:
import time
import torch

def example():
    print('=' * 60)
    print('🚀 A100 PERFORMANCE TEST: Qwen-2.5-3B (Bfloat16 + Flash Attention)')
    print('=' * 60)

    # ---------------------------------------------------------
    # LOAD MODEL
    # ---------------------------------------------------------
    print('\n[1/4] Loading Model...')
    start_load = time.time()

    generator = LLM(model_id="Qwen/Qwen2.5-3B-Instruct", use_quantization=False)

    end_load = time.time()
    print(f'✅ Model Loaded in: {end_load - start_load:.2f}s')
    if torch.cuda.is_available():
        print(f'💾 VRAM Allocation: {torch.cuda.memory_allocated()/1024**3:.2f} GB')

    # ---------------------------------------------------------
    # SINGLE INFERENCE TEST
    # ---------------------------------------------------------
    print('\n[2/4] Testing Single Inference (Normal Chat)...')
    print('-' * 40)

    user_input = "Giải thích ngắn gọn định luật bảo toàn năng lượng."

    start_single = time.time()
    result = generator.generate_response(user_input)
    end_single = time.time()

    print(f"❓ Input:  {user_input}")
    print(f"💡 Output: {result}")
    print(f"⏱️ Time:   {end_single - start_single:.2f}s")

    # ---------------------------------------------------------
    # BATCH INFERENCE TEST
    # ---------------------------------------------------------
    print('\n[3/4] Testing Batch Inference (High Throughput)...')
    print("👉 Kịch bản: Xử lý song song 5 câu hỏi cùng lúc.")
    print('-' * 40)

    batch_prompts = [
        "Thủ đô của Pháp là gì?",
        "Viết công thức tính diện tích hình tròn.",
        "Dịch 'Hello world' sang tiếng Việt.",
        "Liệt kê 3 màu cơ bản.",
        "Năm 2024 là năm con gì?"
    ]

    start_batch = time.time()
    # Gọi hàm generate_batch mới thêm vào
    batch_results = generator.generate_batch(batch_prompts, system_prompt="Bạn là trợ lý ngắn gọn.")
    end_batch = time.time()

    total_time = end_batch - start_batch

    for i, (inp, out) in enumerate(zip(batch_prompts, batch_results)):
        print(f"  [{i+1}] Q: {inp} | A: {out.strip()}")

    print('-' * 40)
    print(f"⏱️ Total Batch Time: {total_time:.2f}s (cho {len(batch_prompts)} câu)")
    print(f"⚡ Average Speed:    {total_time / len(batch_prompts):.2f}s/câu")
    print(">> Nhận xét: Batch size càng lớn, thời gian trung bình/câu sẽ càng giảm trên A100.")

    # ---------------------------------------------------------
    # CLEANUP
    # ---------------------------------------------------------
    print('\n[4/4] Cleanup...')
    generator.unload_model()
    print('=' * 60)
    print('DONE.')

In [5]:
example()

🚀 A100 PERFORMANCE TEST: Qwen-2.5-3B (Bfloat16 + Flash Attention)

[1/4] Loading Model...
>> 📥 Loading Model: Qwen/Qwen2.5-3B-Instruct...
>> 🚀 Hardware Optimization: A100 Mode (Flash Attention 2 + bfloat16)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

>> ✅ Model Loaded Successfully!
>> 💾 VRAM Used: 5.75 GiB
✅ Model Loaded in: 26.41s
💾 VRAM Allocation: 5.75 GB

[2/4] Testing Single Inference (Normal Chat)...
----------------------------------------
❓ Input:  Giải thích ngắn gọn định luật bảo toàn năng lượng.
💡 Output: Định luật bảo toàn năng lượng là một nguyên tắc cơ bản trong vật lý, nói rằng tổng lượng năng lượng trong một hệ thống không thay đổi theo thời gian, tức là năng lượng không bao giờ được tạo ra hoặc phá hủy mà chỉ chuyển đổi từ một dạng thành dạng khác. Điều này có nghĩa là tổng số năng lượng ban đầu trong hệ thống luôn bằng tổng số năng lượng cuối cùng sau khi quá trình diễn ra.
⏱️ Time:   6.60s

[3/4] Testing Batch Inference (High Throughput)...
👉 Kịch bản: Xử lý song song 5 câu hỏi cùng lúc.
----------------------------------------
  [1] Q: Thủ đô của Pháp là gì? | A: Thủ đô của Pháp là Paris.
  [2] Q: Viết công thức tính diện tích hình tròn. | A: Diện tích hình tròn được tính bằng công thức: πr², trong đó r là bán k

## 2. Prompt Preprocessor

### 2.1. Preprocessor Definition

In [21]:
def load_configs_preprocessor():
    return {
        "PromptPreprocessor": {
            "MODEL_NAME": "Qwen/Qwen2.5-3B-Instruct",
            "MAX_TOKENS": 512,
            "MAX_INPUT_LENGTH": 1024,
            "TEMPERATURE": 0.1,
            "TOP_P": 0.95,
            "BATCH_SIZE": 256
        }
    }

In [24]:
class PromptPreprocessor:
    # Định nghĩa System Prompt chuẩn
    DEFAULT_SYSTEM_PROMPT = (
        "Bạn là một trợ lý AI chuyên sửa lỗi chính tả tiếng Việt.\n"
        "Nhiệm vụ: Chuyển đổi câu sai chính tả thành câu đúng chuẩn ngữ pháp và dấu câu.\n"
        "Yêu cầu tuyệt đối:\n"
        "1. Giữ nguyên ý nghĩa và số lượng từ của câu gốc.\n"
        "2. Chỉ sửa lỗi chính tả, teencode (vd: 'k' -> 'không', 'j' -> 'gì').\n"
        "3. KHÔNG thêm bớt từ, KHÔNG giải thích, chỉ trả về câu đã sửa."
    )

    def __init__(self, config: Dict):
        """
        Khởi tạo Preprocessor và load LLM dựa trên config.
        """
        pp_config = config.get('PromptPreprocessor', {})

        # Lấy thông tin model từ config
        model_name = pp_config.get('MODEL_NAME', "Qwen/Qwen2.5-3B-Instruct")
        use_quant = pp_config.get('USE_QUANTIZATION', False)

        # --- SỬA Ở ĐÂY ---
        # Khởi tạo class LLM backend
        self.llm = LLM(
            model_id=model_name,
            use_quantization=use_quant
        )

        self.system_prompt = pp_config.get('SYSTEM_PROMPT', self.DEFAULT_SYSTEM_PROMPT)
        self.BATCH_SIZE = pp_config.get('BATCH_SIZE', 4) # Lấy batch size từ config

    # =========================================================
    # CORE LOGIC
    # =========================================================

    def _is_semantically_preserved(self, inp: str, out: str) -> bool:
        """Giữ nguyên logic kiểm tra độ lệch từ của bạn"""
        inp_words = inp.strip().split()
        out_words = out.strip().split()

        len_in = len(inp_words)
        len_out = len(out_words)

        if len_in == 0: return False
        if abs(len_in - len_out) > 2: # Logic cũ của bạn
            return False

        return True

    def _correct_batch(self, raw_texts: List[str]) -> List[str]:
        """
        --- SỬA QUAN TRỌNG Ở ĐÂY ---
        Thay vì tự viết lại _generate_batch gây lỗi self.model không tìm thấy,
        hãy gọi trực tiếp hàm generate_batch của class LLM.
        """
        # Gọi sang class LLM để xử lý (đã bao gồm chat template + tokenize + generate)
        outputs = self.llm.generate_batch(raw_texts, self.system_prompt)

        final_results = []
        for inp, out in zip(raw_texts, outputs):
            if out and self._is_semantically_preserved(inp, out):
                final_results.append(out)
            else:
                final_results.append(inp)
        return final_results

    # ================================================================
    # CORE BUILD-IN METHODS (ĐÃ KHÔI PHỤC)
    # ================================================================
    def run_example(
        self,
        sample_prompt=[
            "Th3 nao la cay an qa",
            "ngay hum nay troi dep qua minh di choi nhe",
            "ê pà owy cái này dùq xao dọ"
        ]
    ):
        import time

        # Lấy model name từ self.llm.model_id thay vì self.MODEL_NAME (biến này không tồn tại)
        print(f'=== CONFIG ===\nModel: {self.llm.model_id}\nBatch Size: {self.BATCH_SIZE}')

        starting = time.time()
        outputs = self._correct_batch(sample_prompt)
        ending = time.time()
        duration = ending - starting

        for i, (inp, out) in enumerate(zip(sample_prompt, outputs), 1):
            print(f"{i}. IN : {inp}")
            print(f"   OUT: {out}")
            print("-" * 20)

        print(f'Total Time: {duration:.2f}s')
        if duration > 0:
            print(f'Speed: {len(sample_prompt) / duration:.2f} sentences/sec')

        return None

    def process(self, dataset: pd.DataFrame, input_column: str, output_column: str, batch_size: int = None):
        """Logic chạy Batch với Pandas giữ nguyên của bạn"""
        bs = batch_size if batch_size else self.BATCH_SIZE

        df = dataset.copy()
        if input_column not in dataset.columns:
            raise ValueError(f"Input column '{input_column}' not found.")

        df.dropna(subset=[input_column], inplace=True)
        # Sửa nhỏ: convert to string để tránh lỗi nếu cột input có số
        df[input_column] = df[input_column].astype(str)

        all_data = df[input_column].tolist()
        all_corrected_text = []

        print(f"Processing {len(df)} rows with batch_size={bs}...")
        # Thêm tqdm import nếu chưa có ở ngoài
        from tqdm import tqdm

        for i in tqdm(range(0, len(all_data), bs), desc="Batch Processing"):
            batch_texts = all_data[i : i + bs]
            try:
                batch_results = self._correct_batch(batch_texts)
                all_corrected_text.extend(batch_results)
            except Exception as e:
                print(f"[Error] Batch {i} failed: {e}")
                all_corrected_text.extend(batch_texts)

        if len(all_corrected_text) == len(df):
            df[output_column] = all_corrected_text
        return df

    def close(self):
        """Hàm đóng model"""
        self.llm.unload_model()

### 2.2. Preprocessor Usage

In [25]:
prompt_preprocessor = PromptPreprocessor(load_configs_preprocessor())
prompt_preprocessor.run_example()

>> 📥 Loading Model: Qwen/Qwen2.5-3B-Instruct...
>> 🚀 Hardware Optimization: A100 Mode (Flash Attention 2 + bfloat16)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

>> ✅ Model Loaded Successfully!
>> 💾 VRAM Used: 9.58 GiB
=== CONFIG ===
Model: Qwen/Qwen2.5-3B-Instruct
Batch Size: 256
1. IN : Th3 nao la cay an qa
   OUT: Th3 nào là cây ăn quả
--------------------
2. IN : ngay hum nay troi dep qua minh di choi nhe
   OUT: ngày hôm nay trời đẹp quá mình đi chơi nhá
--------------------
3. IN : ê pà owy cái này dùq xao dọ
   OUT: À ông ấy đây này có sao không?
--------------------
Total Time: 1.03s
Speed: 2.92 sentences/sec


## 3. Main pipeline

### 3.1. Load dataset

In [17]:
def load_dataset():
    print('> Start loading dataset...')

    file_ids = {
        'train': '1s7kEVugop7Gv_iws5Kv7OyvLnoRVzjCA',
        'dev':   '1IrCff4cReAbdonve5__9fR3FC105d-LP',
        'test':  '1NPWG2KVeKQ6FPck851wCobPxDJIU3l5_'
    }

    base_url = 'https://drive.google.com/uc?id='
    dfs = {}
    try:
        for name, file_id in file_ids.items():
            dl_url = f"{base_url}{file_id}"
            print(f'  - Downloading & Reading {name} data...')
            dfs[name] = pd.read_csv(dl_url)

        print('> ✅ Dataset loaded successfully.')
        return dfs['train'], dfs['dev'], dfs['test']

    except Exception as e:
        print(f"\n❌ Lỗi khi tải dữ liệu: {e}")
        print("Mẹo: Hãy chắc chắn rằng link đã bật chế độ 'Anyone with the link' (Public).")
        return None, None, None

### 3.2. Execute pipeline

In [30]:
def main(train_df, dev_df, test_df):
    # ---------------------------------------------------------
    # 1. SETUP: Cấu hình và Khởi tạo Preprocessor
    # ---------------------------------------------------------
    print(">> 🚀 Initializing PromptPreprocessor (Batch Size: 128)...")

    config = {
        'PromptPreprocessor': {
            'MODEL_NAME': "Qwen/Qwen2.5-3B-Instruct",
            'BATCH_SIZE': 128,      # Theo yêu cầu
            'USE_QUANTIZATION': False # Set True nếu VRAM yếu, False nếu dùng A100
        }
    }

    # Khởi tạo class (Nó sẽ tự load model)
    preprocessor = PromptPreprocessor(load_configs_preprocessor())

    # ---------------------------------------------------------
    # 2. PREPARE DATASETS
    # ---------------------------------------------------------
    datasets = []
    if train_df is not None: datasets.append(("train", train_df))
    if dev_df is not None:   datasets.append(("dev", dev_df))
    if test_df is not None:  datasets.append(("test", test_df))

    if not datasets:
        print("❌ Không có dữ liệu đầu vào.")
        return

    # Tạo thư mục lưu kết quả
    output_dir = 'data/processed'
    os.makedirs(output_dir, exist_ok=True)

    # ---------------------------------------------------------
    # 3. PROCESSING LOOP
    # ---------------------------------------------------------
    for name, df in datasets:
        print('\n' + '=' * 50)
        print(f"Dataset: [{name}] | Shape: {df.shape}")
        print('=' * 50)

        # --- GỌI HÀM PROCESS CỦA CLASS ---
        # Hàm này đã bao gồm logic chia batch, progress bar và try-catch
        # Input: 'prompt' -> Output: 'corrected_prompt'
        result_df = preprocessor.process(
            dataset=df,
            input_column='prompt',        # Tên cột đầu vào (User yêu cầu)
            output_column='corrected_prompt', # Tên cột đầu ra
            batch_size=128                # User yêu cầu
        )

        # ---------------------------------------------------------
        # 4. SAVE RESULTS
        # ---------------------------------------------------------
        output_path = f'{output_dir}/corrected_{name}.csv'
        print(f'>> 💾 Saving to {output_path}...')
        result_df.to_csv(output_path, index=False)

    # ---------------------------------------------------------
    # 5. CLEANUP & DOWNLOAD
    # ---------------------------------------------------------
    # Giải phóng VRAM
    preprocessor.close()

    print("\n>> 📦 Zipping and downloading results...")
    try:
        os.system(f"zip -r processed_data.zip {output_dir}")
        files.download('processed_data.zip')
    except Exception as e:
        print(f"⚠️ Không thể download tự động: {e}")
        print(f"File đã được lưu tại: {output_dir}")

In [32]:
train, dev, test = load_dataset()

> Start loading dataset...
  - Downloading & Reading train data...
  - Downloading & Reading dev data...
  - Downloading & Reading test data...
> ✅ Dataset loaded successfully.


In [33]:
'''
train = train.head(5)
dev = dev.head(5)
test = test.head(5)
'''

main(train, dev, test)

>> 🚀 Initializing PromptPreprocessor (Batch Size: 128)...
>> 📥 Loading Model: Qwen/Qwen2.5-3B-Instruct...
>> 🚀 Hardware Optimization: A100 Mode (Flash Attention 2 + bfloat16)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

>> ✅ Model Loaded Successfully!
>> 💾 VRAM Used: 22.25 GiB

Dataset: [train] | Shape: (5600, 5)
Processing 5600 rows with batch_size=128...


Batch Processing: 100%|██████████| 44/44 [06:55<00:00,  9.45s/it]


>> 💾 Saving to data/processed/corrected_train.csv...

Dataset: [dev] | Shape: (700, 5)
Processing 700 rows with batch_size=128...


...

>> 💾 Saving to data/processed/corrected_dev.csv...

Dataset: [test] | Shape: (700, 5)
Processing 700 rows with batch_size=128...


Batch Processing: 100%|██████████| 6/6 [00:47<00:00,  7.95s/it]


>> 💾 Saving to data/processed/corrected_test.csv...
>> 🗑️ Unloading model Qwen/Qwen2.5-3B-Instruct...
>> ✅ Memory cleared!

>> 📦 Zipping and downloading results...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>